In [12]:
from d3s.algorithms import gedmd
from d3s.domain import discretization
from d3s.observables import gaussians_with_id
import torch 
from molecular_mpns.proto_molecule import Molecule
from molecular_mpns.data import MarkovMolGraph
from molecular_mpns.config import data_dir,fig_dir,model_dir
from molecular_mpns.modules import ProtoNet
from molecular_mpns.math import proto_reference_params
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# set up molecular environment 
N = 5
beta = 15.0
kb = 1.0
rb = 2.0
ka = 1.0
ra = np.pi/2
kd = np.array([[0.02, 3], [0.02, 2]])
mol = Molecule(N, beta, kb, rb, ka, ra, kd)

# load data
data = np.load(str(data_dir)+'/proto_mol_traj.npy')

# training-validation partition
np.random.seed(42)
part = 1000000
n_train,n_val = 1000000,1000000

train,val = data[0:part],data[part:]
train_idx,val_idx = np.random.choice(train.shape[0],n_train),np.random.choice(val.shape[0],n_val)

In [14]:
# gedmd params
bounds,boxes = np.array([[-np.pi,np.pi],[-np.pi,np.pi]]),np.array([10,10])
omega = discretization(bounds,boxes)
sigma = np.sqrt(0.05)
psi = gaussians_with_id(Omega = omega,sigma = sigma)

# dihedral angle coarse-grained reference model

X = np.zeros((2,n_train))
Y = np.zeros((2,n_train))
Z = np.zeros((2,3*N,n_train))

for i,idx in enumerate(train_idx):
    a_coords = train[idx]
    x = mol._internal_coordinates(a_coords)[7:9]
    dV = mol._gradient(a_coords)
    y,z = proto_reference_params(torch.tensor(a_coords),dV,beta)
    X[:,i] = x
    Y[:,i] = y.reshape(2)
    Z[:,:,i] = z

A,d,V = gedmd(X,-Y,-Z,psi,diffusion = True,evs=10)